Interacting with histories in Galaxy API
========================================

We are going to use the [requests](http://python-requests.org/) Python library to communicate via HTTP with the Galaxy server. To start, let's define the connection parameters.

**You need to insert the API key for your Galaxy server in the cell below**: 
1. Open the Galaxy server in another browser tab
2. Click on "User" on the top menu, then "Preferences"
3. Click on "Manage API key"
4. Generate an API key if needed, then copy the alphanumeric string and paste it as the value of the `api_key` variable below.

In [1]:
from __future__ import print_function

import json

import requests
from six.moves.urllib.parse import urljoin

server = 'https://usegalaxy.org/'
api_key = '1a6625037e2a986f36dc17e0324743fa'
base_url = urljoin(server, 'api')
base_url

'https://usegalaxy.org/api'

We now make a GET request to retrieve all histories owned by a user:

In [2]:
params = {'key': api_key}
r = requests.get(base_url + '/histories', params)
print(r.text)
hists = r.json()
hists

[{"name": "New history", "tags": [], "deleted": false, "purged": false, "annotation": null, "url": "/api/histories/c5d5276103ac10f1", "published": false, "model_class": "History", "id": "c5d5276103ac10f1"}, {"name": "New history", "tags": [], "deleted": false, "purged": false, "annotation": null, "url": "/api/histories/435930ea78663f6f", "published": false, "model_class": "History", "id": "435930ea78663f6f"}, {"name": "New history", "tags": [], "deleted": false, "purged": false, "annotation": null, "url": "/api/histories/153c3c9e6e15ceaa", "published": false, "model_class": "History", "id": "153c3c9e6e15ceaa"}, {"name": "Unnamed history", "tags": [], "deleted": false, "purged": false, "annotation": null, "url": "/api/histories/ccaaa3957982cf4b", "published": false, "model_class": "History", "id": "ccaaa3957982cf4b"}]


[{u'annotation': None,
  u'deleted': False,
  u'id': u'c5d5276103ac10f1',
  u'model_class': u'History',
  u'name': u'New history',
  u'published': False,
  u'purged': False,
  u'tags': [],
  u'url': u'/api/histories/c5d5276103ac10f1'},
 {u'annotation': None,
  u'deleted': False,
  u'id': u'435930ea78663f6f',
  u'model_class': u'History',
  u'name': u'New history',
  u'published': False,
  u'purged': False,
  u'tags': [],
  u'url': u'/api/histories/435930ea78663f6f'},
 {u'annotation': None,
  u'deleted': False,
  u'id': u'153c3c9e6e15ceaa',
  u'model_class': u'History',
  u'name': u'New history',
  u'published': False,
  u'purged': False,
  u'tags': [],
  u'url': u'/api/histories/153c3c9e6e15ceaa'},
 {u'annotation': None,
  u'deleted': False,
  u'id': u'ccaaa3957982cf4b',
  u'model_class': u'History',
  u'name': u'Unnamed history',
  u'published': False,
  u'purged': False,
  u'tags': [],
  u'url': u'/api/histories/ccaaa3957982cf4b'}]

As you can see, GET requests in Galaxy API return JSON strings, which need to be **deserialized** into Python data structures. In particular, GETting a resource collection returns a list of dictionaries.

Each dictionary returned when GETting a resource collection gives basic info about a resource, e.g. for a history you have:
- `id`: the unique **identifier** of the history, needed for all specific requests about this resource
- `name`: the name of this history as given by the user
- `deleted`: whether the history has been deleted
- `url`: the relative URL to get all info about this resource.

There is no readily-available filtering capability, but it's not difficult to filter histories **by name**:

In [3]:
[_ for _ in hists if _['name'] == 'Unnamed history']

[{u'annotation': None,
  u'deleted': False,
  u'id': u'ccaaa3957982cf4b',
  u'model_class': u'History',
  u'name': u'Unnamed history',
  u'published': False,
  u'purged': False,
  u'tags': [],
  u'url': u'/api/histories/ccaaa3957982cf4b'}]

If you are interested in more **details** about a given resource, you just need to append its `id` to the previous collection request, e.g. to the get more info for a history:

In [4]:
hist0_id = hists[0]['id']
print(hist0_id)
params = {'key': api_key}
r = requests.get(base_url + '/histories/' + hist0_id, params)
r.json()

c5d5276103ac10f1


{u'annotation': None,
 u'contents_url': u'/api/histories/c5d5276103ac10f1/contents',
 u'create_time': u'2019-07-02T14:48:13.162661',
 u'deleted': False,
 u'empty': False,
 u'genome_build': None,
 u'id': u'c5d5276103ac10f1',
 u'importable': False,
 u'model_class': u'History',
 u'name': u'New history',
 u'published': False,
 u'purged': False,
 u'size': 0,
 u'slug': None,
 u'state': u'queued',
 u'state_details': {u'discarded': 0,
  u'empty': 0,
  u'error': 0,
  u'failed_metadata': 0,
  u'new': 1,
  u'ok': 0,
  u'paused': 0,
  u'queued': 1,
  u'running': 0,
  u'setting_metadata': 0,
  u'upload': 0},
 u'state_ids': {u'discarded': [],
  u'empty': [],
  u'error': [],
  u'failed_metadata': [],
  u'new': [u'bbd44e69cb8906b542d15e93bc4cf528'],
  u'ok': [],
  u'paused': [],
  u'queued': [u'bbd44e69cb8906b569ca5e103d37687f'],
  u'running': [],
  u'setting_metadata': [],
  u'upload': []},
 u'tags': [],
 u'update_time': u'2019-07-02T14:48:39.899514',
 u'url': u'/api/histories/c5d5276103ac10f1',
 u'u

As you can see, there are much more entries in the returned dictionary, e.g.:
- `create_time`
- `size`: total disk space used by the history
- `state_ids`: ids of history datasets for each possible state.

To get the list of **datasets contained** in a history, simply append `/contents` to the previous resource request.

In [5]:
params = {'key': api_key}
r = requests.get(base_url + '/histories/' + hist0_id + '/contents', params)
hdas = r.json()
hdas

[{u'create_time': u'2019-07-02T14:48:18.739845',
  u'dataset_id': u'bbd44e69cb8906b5c74c308e0955b174',
  u'deleted': False,
  u'extension': u'auto',
  u'hid': 1,
  u'history_content_type': u'dataset',
  u'history_id': u'c5d5276103ac10f1',
  u'id': u'bbd44e69cb8906b569ca5e103d37687f',
  u'name': u'1.txt',
  u'purged': False,
  u'state': u'queued',
  u'tags': [],
  u'type': u'file',
  u'type_id': u'dataset-bbd44e69cb8906b569ca5e103d37687f',
  u'update_time': u'2019-07-02T14:48:18.896395',
  u'url': u'/api/histories/c5d5276103ac10f1/contents/bbd44e69cb8906b569ca5e103d37687f',
  u'visible': True},
 {u'create_time': u'2019-07-02T14:48:39.831910',
  u'dataset_id': u'bbd44e69cb8906b5aeb2fb6459df02f2',
  u'deleted': False,
  u'extension': u'tabular',
  u'hid': 2,
  u'history_content_type': u'dataset',
  u'history_id': u'c5d5276103ac10f1',
  u'id': u'bbd44e69cb8906b542d15e93bc4cf528',
  u'name': u'Convert on data 1',
  u'purged': False,
  u'state': u'new',
  u'tags': [],
  u'type': u'file',
  u

The dictionaries returned when GETting the history content give basic info about each dataset, e.g.: `id`, `name`, `deleted`, `state`, `url`...

To get the details about a specific dataset, you can use the `datasets` controller:

In [6]:
hda0_id = hdas[0]['id']
print(hda0_id)
params = {'key': api_key}
r = requests.get(base_url + '/datasets/' + hda0_id, params)
r.json()

bbd44e69cb8906b569ca5e103d37687f


{u'accessible': True,
 u'annotation': None,
 u'api_type': u'file',
 u'create_time': u'2019-07-02T14:48:18.739845',
 u'creating_job': u'bbd44e69cb8906b58fd9ba7eb07afdd8',
 u'data_type': u'galaxy.datatypes.data.Data',
 u'dataset_id': u'bbd44e69cb8906b5c74c308e0955b174',
 u'deleted': False,
 u'display_apps': [],
 u'display_types': [],
 u'download_url': u'/api/histories/c5d5276103ac10f1/contents/bbd44e69cb8906b569ca5e103d37687f/display',
 u'extension': u'auto',
 u'file_ext': u'auto',
 u'file_size': 0,
 u'genome_build': u'?',
 u'hda_ldda': u'hda',
 u'hid': 1,
 u'history_content_type': u'dataset',
 u'history_id': u'c5d5276103ac10f1',
 u'id': u'bbd44e69cb8906b569ca5e103d37687f',
 u'meta_files': [],
 u'metadata_dbkey': u'?',
 u'misc_blurb': None,
 u'misc_info': None,
 u'model_class': u'HistoryDatasetAssociation',
 u'name': u'1.txt',
 u'peek': None,
 u'purged': False,
 u'rerunnable': False,
 u'resubmitted': False,
 u'state': u'queued',
 u'tags': [],
 u'type': u'file',
 u'type_id': u'dataset-bbd

Some of the interesting additional dictionary entries are:
- `create_time`
- `creating job`: id of the job which created this dataset
- `download_url`: URL to download the dataset
- `file_ext`: the Galaxy data type of this dataset
- `file_size`
- `genome_build`: the genome build (dbkey) associated to this dataset.

**New resources** are created with POST requests. The uploaded **data needs to be serialized** in a JSON string. For example, to create a new history:

In [7]:
params = {'key': api_key}
data = {'name': 'New history'}
r = requests.post(base_url + '/histories', data=json.dumps(data), params=params, headers={'Content-Type': 'application/json'})
new_hist = r.json()
new_hist

{u'annotation': None,
 u'contents_url': u'/api/histories/4c8f0af535599c16/contents',
 u'create_time': u'2019-07-02T14:49:15.151782',
 u'deleted': False,
 u'empty': True,
 u'genome_build': None,
 u'id': u'4c8f0af535599c16',
 u'importable': False,
 u'model_class': u'History',
 u'name': u'New history',
 u'published': False,
 u'purged': False,
 u'size': 0,
 u'slug': None,
 u'state': u'new',
 u'state_details': {u'discarded': 0,
  u'empty': 0,
  u'error': 0,
  u'failed_metadata': 0,
  u'new': 0,
  u'ok': 0,
  u'paused': 0,
  u'queued': 0,
  u'running': 0,
  u'setting_metadata': 0,
  u'upload': 0},
 u'state_ids': {u'discarded': [],
  u'empty': [],
  u'error': [],
  u'failed_metadata': [],
  u'new': [],
  u'ok': [],
  u'paused': [],
  u'queued': [],
  u'running': [],
  u'setting_metadata': [],
  u'upload': []},
 u'tags': [],
 u'update_time': u'2019-07-02T14:49:15.151793',
 u'url': u'/api/histories/4c8f0af535599c16',
 u'user_id': u'2841a31c8997a2d3',
 u'username_and_slug': None}

The return value of a POST request is a dictionary with detailed info about the created resource.

To **update** a resource, make a PUT request, e.g. to change the history name:

In [8]:
params = {'key': api_key}
data = {'name': 'Updated history'}
r = requests.put(base_url + '/histories/' + new_hist['id'], json.dumps(data), params=params, headers={'Content-Type': 'application/json'})
print(r.status_code)
r.json()

200


{u'annotation': None,
 u'contents_url': u'/api/histories/4c8f0af535599c16/contents',
 u'create_time': u'2019-07-02T14:49:15.151782',
 u'deleted': False,
 u'empty': True,
 u'genome_build': None,
 u'id': u'4c8f0af535599c16',
 u'importable': False,
 u'model_class': u'History',
 u'name': u'Updated history',
 u'published': False,
 u'purged': False,
 u'size': 0,
 u'slug': None,
 u'state': u'new',
 u'state_details': {u'discarded': 0,
  u'empty': 0,
  u'error': 0,
  u'failed_metadata': 0,
  u'new': 0,
  u'ok': 0,
  u'paused': 0,
  u'queued': 0,
  u'running': 0,
  u'setting_metadata': 0,
  u'upload': 0},
 u'state_ids': {u'discarded': [],
  u'empty': [],
  u'error': [],
  u'failed_metadata': [],
  u'new': [],
  u'ok': [],
  u'paused': [],
  u'queued': [],
  u'running': [],
  u'setting_metadata': [],
  u'upload': []},
 u'tags': [],
 u'update_time': u'2019-07-02T14:49:18.517751',
 u'url': u'/api/histories/4c8f0af535599c16',
 u'user_id': u'2841a31c8997a2d3',
 u'username_and_slug': None}

The return value of a PUT request is usually a dictionary with detailed info about the updated resource.

Finally to **delete** a resource, make a DELETE request, e.g.:

In [9]:
params = {'key': api_key}
r = requests.delete(base_url + '/histories/' + new_hist['id'], params=params)
print(r.status_code)

200
